# Import Dependencies

In [178]:
import pandas as pd # Data Manipulation
import numpy as np # Linear Algebra


#Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Prepcrocessing


# Others
import os
import warnings
warnings.filterwarnings('ignore')

# Load Data

In [179]:
data ={}

In [180]:
path = 'raw_data'

for i in os.listdir(path):
    subfolder = str(i)
    subfolder = os.path.join(path,subfolder)
    for j in os.listdir(subfolder):
        dict_key = j[:-4]
        if "TeamSpellings" not in j:
            data[dict_key] = pd.read_csv(os.path.join(subfolder,j))
        else:
            data[dict_key] = pd.read_csv(os.path.join(subfolder,j), encoding='cp1252')

In [181]:
MNCAATourneyCompactResults = data['MNCAATourneyCompactResults']
MNCAATourneySeeds = data['MNCAATourneySeeds']
MRegularSeasonCompactResults = data['MRegularSeasonCompactResults']
MSeasons = data['MSeasons']
MTeams = data['MTeams']
SampleSubmission2023 = data['SampleSubmission2023']
WNCAATourneyCompactResults = data['WNCAATourneyCompactResults']
WNCAATourneySeeds = data['WNCAATourneySeeds']
WRegularSeasonCompactResults = data['WRegularSeasonCompactResults']
WSeasons = data['WSeasons']
WTeams = data['WTeams']
MNCAATourneyDetailedResults = data['MNCAATourneyDetailedResults']
MRegularSeasonDetailedResults = data['MRegularSeasonDetailedResults']
WNCAATourneyDetailedResults = data['WNCAATourneyDetailedResults']
WRegularSeasonDetailedResults = data['WRegularSeasonDetailedResults']
Cities = data['Cities']
MGameCities = data['MGameCities']
WGameCities = data['WGameCities']
MMasseyOrdinals = data['MMasseyOrdinals']
Conferences = data['Conferences']
MConferenceTourneyGames = data['MConferenceTourneyGames']
MNCAATourneySeedRoundSlots = data['MNCAATourneySeedRoundSlots']
MNCAATourneySlots = data['MNCAATourneySlots']
MSecondaryTourneyCompactResults = data['MSecondaryTourneyCompactResults']
MSecondaryTourneyTeams = data['MSecondaryTourneyTeams']
MTeamCoaches = data['MTeamCoaches']
MTeamConferences = data['MTeamConferences']
MTeamSpellings = data['MTeamSpellings']
WNCAATourneySlots = data['WNCAATourneySlots']
WTeamConferences = data['WTeamConferences']
WTeamSpellings = data['WTeamSpellings']

# Feature Selection

In [182]:
df_seeds = pd.concat([
    MNCAATourneySeeds,
    WNCAATourneySeeds], ignore_index=True
)
df_seeds.head()

,Season,Seed,TeamID
0,1985,W01,1207
1,1985,W02,1210
2,1985,W03,1228
3,1985,W04,1260
4,1985,W05,1374


In [183]:
# Get season results and drop unnecessary columns

df_season_results = pd.concat([
    MRegularSeasonDetailedResults,
    WRegularSeasonDetailedResults,
], ignore_index=True)

df_season_results.drop(['NumOT', 'WLoc'], axis=1, inplace=True)
df_season_results['ScoreGap'] = df_season_results['WScore'] - df_season_results['LScore']
df_season_results.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WFGM,WFGA,WFGM3,WFGA3,...,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF,ScoreGap
0,2003,10,1104,68,1328,62,27,58,3,14,...,16,22,10,22,8,18,9,2,20,6
1,2003,10,1272,70,1393,63,26,62,8,20,...,9,20,20,25,7,12,8,6,16,7
2,2003,11,1266,73,1437,61,24,58,8,18,...,14,23,31,22,9,12,2,5,23,12
3,2003,11,1296,56,1457,50,18,38,3,9,...,8,15,17,20,9,19,4,3,23,6
4,2003,11,1400,77,1208,71,30,61,6,14,...,17,27,21,15,12,10,7,1,14,6


In [184]:
num_win = df_season_results.groupby(['Season', 'WTeamID']).count()
num_win = num_win.reset_index()[['Season', 'WTeamID', 'DayNum']].rename(columns={"DayNum": "NumWins", "WTeamID": "TeamID"})

In [185]:
num_loss = df_season_results.groupby(['Season', 'LTeamID']).count()
num_loss = num_loss.reset_index()[['Season', 'LTeamID', 'DayNum']].rename(columns={"DayNum": "NumLosses", "LTeamID": "TeamID"})

In [186]:
num_win = num_win.sort_values('NumWins',ascending=False)
num_win.head()

,Season,TeamID,NumWins
5927,2015,1246,34
5497,2014,3163,34
1811,2008,1272,33
4080,2012,3124,33
5432,2014,1455,33


In [187]:
num_loss = num_loss.sort_values('NumLosses',ascending=True)
num_loss.head()

,Season,TeamID,NumLosses
7295,2017,1211,1
10395,2021,3163,1
3707,2011,3453,1
4960,2013,3323,1
6896,2016,3161,1


In [188]:
gap_win = df_season_results.groupby(['Season', 'WTeamID']).mean().reset_index()
gap_win = gap_win[['Season', 'WTeamID', 'ScoreGap']].rename(columns={"ScoreGap": "GapWins", "WTeamID": "TeamID"})
gap_win.sort_values('GapWins')
gap_win.head()

,Season,TeamID,GapWins
0,2003,1102,15.583333
1,2003,1103,9.384615
2,2003,1104,13.176471
3,2003,1105,13.000000
4,2003,1106,10.384615


In [189]:
gap_loss = df_season_results.groupby(['Season', 'LTeamID']).mean().reset_index()
gap_loss = gap_loss[['Season', 'LTeamID', 'ScoreGap']].rename(columns={"ScoreGap": "GapLosses", "LTeamID": "TeamID"})
gap_loss.sort_values('GapLosses')
gap_loss.head()

,Season,TeamID,GapLosses
0,2003,1102,11.250000
1,2003,1103,7.500000
2,2003,1104,9.454545
3,2003,1105,11.473684
4,2003,1106,9.266667


In [190]:
df_features_season_w = df_season_results.groupby(['Season', 'WTeamID']).count().reset_index()[['Season', 'WTeamID']].rename(columns={"WTeamID": "TeamID"})
df_features_season_l = df_season_results.groupby(['Season', 'LTeamID']).count().reset_index()[['Season', 'LTeamID']].rename(columns={"LTeamID": "TeamID"})

df_features_season = pd.concat([df_features_season_w, df_features_season_l], axis=0).drop_duplicates().sort_values(['Season', 'TeamID']).reset_index(drop=True)

df_features_season = df_features_season.merge(num_win, on=['Season', 'TeamID'], how='left')
df_features_season = df_features_season.merge(num_loss, on=['Season', 'TeamID'], how='left')
df_features_season = df_features_season.merge(gap_win, on=['Season', 'TeamID'], how='left')
df_features_season = df_features_season.merge(gap_loss, on=['Season', 'TeamID'], how='left')

df_features_season.fillna(0, inplace=True)  
df_features_season.head()

,Season,TeamID,NumWins,NumLosses,GapWins,GapLosses
0,2003,1102,12.0,16.0,15.583333,11.250000
1,2003,1103,13.0,14.0,9.384615,7.500000
2,2003,1104,17.0,11.0,13.176471,9.454545
3,2003,1105,7.0,19.0,13.000000,11.473684
4,2003,1106,13.0,15.0,10.384615,9.266667


In [191]:

df_features_season['WinRatio'] = df_features_season['NumWins'] / (df_features_season['NumWins'] + df_features_season['NumLosses'])
df_features_season['GapAvg'] = (
    (df_features_season['NumWins'] * df_features_season['GapWins'] - 
    df_features_season['NumLosses'] * df_features_season['GapLosses'])
    / (df_features_season['NumWins'] + df_features_season['NumLosses'])
)

df_features_season.drop(['NumWins', 'NumLosses', 'GapWins', 'GapLosses'], axis=1, inplace=True)
df_features_season.head()

,Season,TeamID,WinRatio,GapAvg
0,2003,1102,0.428571,0.250000
1,2003,1103,0.481481,0.629630
2,2003,1104,0.607143,4.285714
3,2003,1105,0.269231,-4.884615
4,2003,1106,0.464286,-0.142857


In [192]:
df_season_results.columns

Index(['Season', 'DayNum', 'WTeamID', 'WScore', 'LTeamID', 'LScore', 'WFGM',
       'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR', 'WAst', 'WTO',
       'WStl', 'WBlk', 'WPF', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3', 'LFTM', 'LFTA',
       'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF', 'ScoreGap'],
      dtype='object')

In [193]:
wins_stats = df_season_results.groupby(['Season','WTeamID']).mean().reset_index()[['Season','WTeamID', 'WScore','WFGM',
       'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR', 'WAst', 'WTO',
       'WStl', 'WBlk', 'WPF']].rename(columns={'WTeamID':'TeamID'})

losses_stats = df_season_results.groupby(['Season','LTeamID']).mean().reset_index()[['Season','LTeamID', 'LScore','LFGM',
       'LFGA', 'LFGM3', 'LFGA3', 'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO',
       'LStl', 'LBlk', 'LPF']].rename(columns={'LTeamID':'TeamID'})

In [194]:
averaged_data = wins_stats.merge(losses_stats, on=['Season','TeamID'])
averaged_data

,Season,TeamID,WScore,WFGM,WFGA,WFGM3,WFGA3,WFTM,WFTA,WOR,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2003,1102,68.750000,22.583333,40.000000,10.000000,21.583333,13.583333,20.750000,3.833333,...,20.250000,9.312500,14.375000,4.437500,14.937500,10.062500,11.687500,4.937500,1.000000,20.750000
1,2003,1103,87.769231,30.000000,55.384615,5.461538,14.384615,22.307692,30.923077,9.384615,...,17.642857,16.000000,21.142857,10.142857,18.500000,12.928571,12.714286,7.214286,2.785714,19.285714
2,2003,1104,74.705882,25.823529,58.352941,7.058824,20.823529,16.000000,22.529412,13.529412,...,18.363636,13.090909,18.454545,13.636364,20.090909,9.181818,13.636364,5.636364,3.181818,20.454545
3,2003,1105,79.428571,25.571429,61.857143,9.142857,22.428571,19.142857,25.714286,14.571429,...,20.157895,14.052632,20.421053,13.105263,22.105263,14.052632,18.894737,8.578947,2.105263,20.526316
4,2003,1106,68.307692,24.769231,53.846154,5.846154,15.923077,12.923077,20.769231,12.769231,...,19.133333,8.666667,12.733333,11.866667,20.266667,10.533333,16.466667,7.466667,2.600000,18.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12089,2023,3473,84.000000,29.000000,47.000000,14.000000,26.000000,12.000000,16.000000,5.000000,...,21.136364,8.500000,12.954545,6.318182,18.818182,11.272727,19.045455,6.272727,1.863636,16.136364
12090,2023,3474,68.600000,23.800000,59.000000,6.800000,20.200000,14.200000,19.200000,10.000000,...,15.555556,11.166667,17.722222,8.111111,19.500000,5.555556,15.111111,6.944444,2.222222,22.055556
12091,2023,3475,71.750000,24.875000,59.375000,5.375000,16.625000,16.625000,21.625000,11.000000,...,15.312500,13.625000,17.750000,7.875000,22.375000,12.687500,21.125000,7.375000,2.437500,19.125000
12092,2023,3476,65.571429,24.285714,56.142857,7.000000,19.000000,10.000000,15.000000,8.428571,...,20.578947,9.473684,12.947368,8.684211,21.315789,13.263158,15.105263,3.684211,2.526316,15.526316


In [195]:
for i in averaged_data.columns:
    if i.startswith('W'):
        averaged_data[i] = (averaged_data[i]+averaged_data['L'+i[1:]])/2
        averaged_data.drop(['L'+i[1:]],axis=1,inplace=True)
        averaged_data = averaged_data.rename(columns = {i:i[1:]+'Avg'})
averaged_data

,Season,TeamID,ScoreAvg,FGMAvg,FGAAvg,FGM3Avg,FGA3Avg,FTMAvg,FTAAvg,ORAvg,DRAvg,AstAvg,TOAvg,StlAvg,BlkAvg,PFAvg
0,2003,1102,58.687500,19.572917,39.812500,8.093750,20.916667,11.447917,17.562500,4.135417,17.135417,13.489583,11.385417,6.135417,1.916667,18.416667
1,2003,1103,79.098901,27.250000,55.835165,5.445055,16.013736,19.153846,26.032967,9.763736,19.980769,15.310440,12.626374,7.260989,2.315934,19.873626
2,2003,1104,67.807487,23.548128,56.858289,6.165775,19.593583,14.545455,20.491979,13.582888,23.251337,11.590909,13.347594,6.435829,3.679144,18.462567
3,2003,1105,74.187970,24.759398,61.691729,8.071429,21.293233,16.597744,23.067669,13.838346,23.981203,14.954887,18.447368,9.932331,2.052632,19.977444
4,2003,1106,63.920513,23.517949,55.189744,6.089744,17.528205,10.794872,16.751282,12.317949,24.133333,11.766667,17.079487,8.425641,3.184615,18.192308
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12089,2023,3473,68.590909,24.159091,48.795455,10.022727,23.568182,10.250000,14.477273,5.659091,21.909091,18.636364,22.022727,7.136364,3.931818,15.568182
12090,2023,3474,61.133333,21.538889,58.333333,5.372222,17.877778,12.683333,18.461111,9.055556,20.950000,7.577778,14.055556,7.772222,2.111111,20.827778
12091,2023,3475,64.750000,22.500000,56.406250,4.625000,15.968750,15.125000,19.687500,9.437500,24.687500,13.718750,19.562500,7.875000,3.093750,18.812500
12092,2023,3476,60.785714,22.327068,55.939850,6.394737,19.789474,9.736842,13.973684,8.556391,23.086466,14.417293,14.552632,4.270677,2.620301,15.120301


In [196]:

df_features_season = df_features_season.merge(averaged_data, on=['Season','TeamID'])
df_features_season

,Season,TeamID,WinRatio,GapAvg,ScoreAvg,FGMAvg,FGAAvg,FGM3Avg,FGA3Avg,FTMAvg,FTAAvg,ORAvg,DRAvg,AstAvg,TOAvg,StlAvg,BlkAvg,PFAvg
0,2003,1102,0.428571,0.250000,58.687500,19.572917,39.812500,8.093750,20.916667,11.447917,17.562500,4.135417,17.135417,13.489583,11.385417,6.135417,1.916667,18.416667
1,2003,1103,0.481481,0.629630,79.098901,27.250000,55.835165,5.445055,16.013736,19.153846,26.032967,9.763736,19.980769,15.310440,12.626374,7.260989,2.315934,19.873626
2,2003,1104,0.607143,4.285714,67.807487,23.548128,56.858289,6.165775,19.593583,14.545455,20.491979,13.582888,23.251337,11.590909,13.347594,6.435829,3.679144,18.462567
3,2003,1105,0.269231,-4.884615,74.187970,24.759398,61.691729,8.071429,21.293233,16.597744,23.067669,13.838346,23.981203,14.954887,18.447368,9.932331,2.052632,19.977444
4,2003,1106,0.464286,-0.142857,63.920513,23.517949,55.189744,6.089744,17.528205,10.794872,16.751282,12.317949,24.133333,11.766667,17.079487,8.425641,3.184615,18.192308
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12089,2023,3473,0.043478,-18.086957,68.590909,24.159091,48.795455,10.022727,23.568182,10.250000,14.477273,5.659091,21.909091,18.636364,22.022727,7.136364,3.931818,15.568182
12090,2023,3474,0.217391,-13.347826,61.133333,21.538889,58.333333,5.372222,17.877778,12.683333,18.461111,9.055556,20.950000,7.577778,14.055556,7.772222,2.111111,20.827778
12091,2023,3475,0.333333,-3.125000,64.750000,22.500000,56.406250,4.625000,15.968750,15.125000,19.687500,9.437500,24.687500,13.718750,19.562500,7.875000,3.093750,18.812500
12092,2023,3476,0.269231,-6.923077,60.785714,22.327068,55.939850,6.394737,19.789474,9.736842,13.973684,8.556391,23.086466,14.417293,14.552632,4.270677,2.620301,15.120301


In [197]:
df_test = SampleSubmission2023.copy(deep=True)
df_test.head()

,ID,Pred
0,2023_1101_1102,0.5
1,2023_1101_1103,0.5
2,2023_1101_1104,0.5
3,2023_1101_1105,0.5
4,2023_1101_1106,0.5


In [198]:
df_test['Year'] = df_test['ID'].apply(lambda x: x.split('_')[0])
df_test['TeamA'] = df_test['ID'].apply(lambda x: x.split('_')[1])
df_test['teamB'] = df_test['ID'].apply(lambda x: x.split('_')[2])

df_test

,ID,Pred,Year,TeamA,teamB
0,2023_1101_1102,0.5,2023,1101,1102
1,2023_1101_1103,0.5,2023,1101,1103
2,2023_1101_1104,0.5,2023,1101,1104
3,2023_1101_1105,0.5,2023,1101,1105
4,2023_1101_1106,0.5,2023,1101,1106
...,...,...,...,...,...
130678,2023_3474_3476,0.5,2023,3474,3476
130679,2023_3474_3477,0.5,2023,3474,3477
130680,2023_3475_3476,0.5,2023,3475,3476
130681,2023_3475_3477,0.5,2023,3475,3477


In [201]:
no_include = ['TeamID']


ValueError: cannot insert RAvgA, already exists